In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point
from scipy.io import savemat
import time


def ellipse_coordinates_1(a, b, nm, U1, U2):
    theta = np.linspace(0, 2*np.pi, 200)
    x = np.linspace(-a, a, 200)
    y = b*(1-(np.abs(x/a)**nm))**(1/nm)
    x = np.concatenate([x, x[::-1]])
    y = np.concatenate([y, -y[::-1]])
    X = x + U1
    Y = y + U2
    return X, Y

    
# Define the dimensions of the rectangular surface
Lx = 50 # length in x-direction
Ly = 50 # length in y-direction

# Define the dimensions of the ellipses
a = 2*0.25 # major axis
b = 0.25 # minor axis
theta = np.linspace(0, 2*np.pi, 200)

# Initialize a list to store the ellipses
ellipses = []

# Load existing ellipses if any
# load('AR1_repeat.mat')

# Define the maximum distance for checking overlap 
max_dist = 2*a

# Define the filename to save the centroids
filename = 'centroids.mat'


# Loop over the number of ellipses to place
for i in range(len(ellipses), 1000000):
    
    # Initialize a flag to indicate whether the ellipse has been successfully placed
    placed = False
    
    # Loop until the ellipse is successfully placed
    while not placed:
        
        # Generate a random position for the center of the ellipse
        x = np.random.uniform(a, Lx - a)
        y = np.random.uniform(b, Ly - b)
        
        # Create a Polygon object for the ellipse
        X, Y = ellipse_coordinates_1(a, b, 16, x, y)
        ellipse = Polygon(np.column_stack((X, Y)))
        
        # Check if the ellipse overlaps with any existing ellipses within a certain distance
        overlap = False
        for j in range(len(ellipses)):
            if Point(ellipse.centroid).distance(Point(ellipses[j].centroid)) <= max_dist and ellipse.intersects(ellipses[j]):
                overlap = True
                break
        
        # If the ellipse does not overlap, add it to the list of ellipses and increment the counter
        if not overlap:
            ellipses.append(ellipse)
            placed = True
            print(f'Ellipses placed: {len(ellipses)}')
            # Save the centroids every 10 seconds
    if len(ellipses) == 3000:
       break
if len(ellipses) == 3000:
   break    
 
# Save the final centroids
centroids = []
for i in range(sum([1 for e in ellipses if e])):
    e = ellipses[i]
    p = Polygon(e)
    x, y = p.centroid.x, p.centroid.y
    centroids.append([1, x, y])
centroids = np.array(centroids)
savemat(filename, {'centroids': centroids})

# Plot the ellipses
fig, ax = plt.subplots(figsize=(10, 10))
for ellipse in ellipses:
    ax.plot(*ellipse.exterior.xy, color='black')
ax.set_xlim(0, Lx)
     

In [ ]:
# Save the final centroids
centroids = []
for i in range(sum([1 for e in ellipses if e])):
    e = ellipses[i]
    p = Polygon(e)
    x, y = p.centroid.x, p.centroid.y
    centroids.append([1, x, y])
centroids = np.array(centroids)
savemat(filename, {'centroids': centroids})

In [ ]:
import numpy as np

# Create mesh with 0.1 step size
x_range = 50  # replace with the desired range
step_size = 0.25/2
x1, y1 = np.meshgrid(np.arange(0, x_range+step_size, step_size), np.arange(0, x_range+step_size, step_size))

threshold = 0.2110

# Set points outside the desired range to NaN
x1[(x1 < a) | (x1 > x_range - a) | (y1 > x_range - b) | (y1 < b)] = np.nan
y1[(x1 < a) | (x1 > x_range - a) | (y1 > x_range - b) | (y1 < b)] = np.nan

# Define bounds for each vertex
v = centroids 
n = v.shape[0]
xmin = np.zeros(n)
xmax = np.zeros(n)
ymin = np.zeros(n)
ymax = np.zeros(n)

aa = 2*a
bb = 2*b
nm=50
idx_matrix = np.zeros(x1.shape, dtype=bool)
for i in range(n):
    x0 = v[i, 1]
    y0 = v[i, 2]
    d = ((x1 - x0) / aa)**nm + ((y1 - y0) / bb)**nm
    x1[d < 1] = np.nan
    y1[d < 1] = np.nan 

In [ ]:
# Find the rows where both x and y are NaN
nan_rows = np.isnan(x1) | np.isnan(y1)

# Remove the rows from x1 and y1 where both values are NaN
x1 = x1[~nan_rows]
y1 = y1[~nan_rows]
     

In [ ]:
import matplotlib.pyplot as plt

# Create figure and axes
fig, ax = plt.subplots(figsize=(10, 10))

# Plot mesh points
ax.scatter(x1, y1, s=0.1)
for ellipse in ellipses:
    ax.plot(*ellipse.exterior.xy, color='black')
# Set x and y limits
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)

# Show plot
plt.show()
     